# Linear Regression Baseline

We've setup different groups for Linear regression:
- num_insts only
- 26 groups (and they sums to num_inst)
- Per-Inst

Every group provide options for non-traced and traced. Also, (approx) MAPE vs non-mape loss group is given.

The metric is calculated using the function inside misc.
The mape, mae and mse are reported for different time groups.

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(''), '../'))

In [2]:
import logging
from compete.TracedLinearRegression import TracedLinearRegression
from compete.TracedPerInstLinearRegression import (
    TracedPerInstLinearRegression,
    TracedPerInstLinearRegressionTorch
)
from dataset.DatasetBuilder import build_dataset
from misc import metric
from toyDb.utils.spv.SpvContext import getDefaultGrammar
from tqdm.contrib.logging import logging_redirect_tqdm
import tabulate

logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

TODO: add a constrastive on different optimized shaders having different trace count but same runtime

use https://pypi.org/project/tabulate/ to show answer

https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook


## 3060 environment

In [3]:
metrics = {}

def do_compute(
  groupName,
  enableTrace,
  useApproxMape,
  trainDataset,
  testDataset
):
  Y_real = None
  Y_pred = None

  if groupName == 'NumInsts':
    regressor = TracedLinearRegression(1, useApproxMape, enableTrace, False)
    regressor.train(trainDataset)

    print(f"- Model coef: {regressor.model.coef_}")
    print(f"- Model intercept: {regressor.model.intercept_}")

    Y_real, Y_pred = regressor.evaluate(testDataset)

  elif groupName == 'AllSpvCategories':
    regressor = TracedLinearRegression(26, useApproxMape, enableTrace, True)
    regressor.train(trainDataset)

    print(f"- Model coef: {regressor.model.coef_}")
    print(f"- Model intercept: {regressor.model.intercept_}")

    Y_real, Y_pred = regressor.evaluate(testDataset)

  elif groupName == 'PerInst':
    grammar = getDefaultGrammar()
    regressor = TracedPerInstLinearRegression(grammar, useApproxMape, enableTrace)
    regressor.train(trainDataset)

    print(f"- Model coef: {regressor.model.coef_}")
    print(f"- Model intercept: {regressor.model.intercept_}")

    Y_real, Y_pred = regressor.evaluate(testDataset)

  return metric.compute_metrics(Y_pred, Y_real)


In [4]:
trainDataset = build_dataset('FragmentPerformanceTracedSnapshotDataset4096-3060', 'train')
testDataset = build_dataset('FragmentPerformanceTracedSnapshotDataset4096-3060', 'test')

for groupName in ['PerInst', 'NumInsts', 'AllSpvCategories']:
  for enableTrace in [True, False]:
    for useApproxMape in [True, False]:
      fullName = f"{groupName}-{'withTrace' if enableTrace else 'withoutTrace'}-{'useApproxMape' if useApproxMape else 'useMSE'}"
      logger.info(f"Running test for {fullName}")

      with logging_redirect_tqdm():
        metrics[fullName] = do_compute(**{
          'groupName': groupName,
          'enableTrace': enableTrace,
          'useApproxMape': useApproxMape,
          'trainDataset': trainDataset,
          'testDataset': testDataset
        })



2023-11-03 12:03:55,958 - __main__ - INFO - Running test for PerInst-withTrace-useApproxMape
2023-11-03 12:03:55,969 - compete.TracedPerInstLinearRegression - INFO - Collected 702 inst names from grammar
2023-11-03 12:04:01,593 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-11-03 12:04:01,596 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = XtVfWD, environmentId = 1
2023-11-03 12:04:21,202 - compete.TracedPerInstLinearRegression - WARNING - Basic block #25 not in traced block list. 
2023-11-03 12:04:21,206 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = 3tKXzc, environmentId = 1
2023-11-03 12:04:21,745 - compete.TracedPerInstLinearRegression - WARNING - Basic block #27 not in traced block list. 
2023-11-03 12:04:21,747 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = tdsGRX, environmentId = 1
2023-11-03 12:04:29,215 - compete.TracedPerInstLinearRegression - WARNING - 

model coef_=[ 1.91070532e-18 -3.51928570e-10  7.01848519e-14 -3.96724043e-11
 -3.10975481e-11  8.00238673e-13 -8.19695332e-13  7.14093238e-12
  4.81394091e-12  7.89141322e-12  8.41292574e-12 -8.90909829e-12
  5.79441226e-13  6.79538890e-12 -3.09513266e-12 -7.93956806e-13
 -6.10991466e-12 -8.92179039e-12 -1.15792022e-12  2.68177472e-12
 -3.95689644e-12 -2.85576012e-12 -4.78681114e-13  5.96815696e-12
 -3.20120319e-12  4.08411803e-12  7.13906538e-12  4.25884371e-12
  2.73031771e-12  4.27502998e-12 -1.14242795e-12 -1.79501151e-12
  3.72783445e-12  8.78412469e-14 -2.47104203e-13  2.87921542e-13
 -1.11147961e-13 -1.57611554e-14 -8.47974035e-14 -6.27635964e-14
 -9.68653326e-16  7.28009233e-14 -1.04315157e-13 -1.46449259e-13
  8.32361523e-14 -1.12789159e-13 -2.72323396e-14 -8.87086628e-14
  4.67059117e-14  2.00439383e-13 -1.32095832e-13  4.32924096e-14
  1.21381312e-14  6.39664645e-14  1.47913799e-13  6.94626648e-14
  6.96892630e-15  3.48909270e-14 -6.17531200e-14  4.30168054e-15
 -3.12692580e

2023-11-03 12:05:47,411 - compete.TracedPerInstLinearRegression - WARNING - Basic block #36 not in traced block list. 
2023-11-03 12:05:47,415 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = DllSWX, environmentId = 1
2023-11-03 12:05:50,982 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-11-03 12:05:50,985 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 1
2023-11-03 12:05:51,627 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-11-03 12:05:51,629 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 1
2023-11-03 12:05:54,339 - compete.TracedPerInstLinearRegression - WARNING - Basic block #21 not in traced block list. 
2023-11-03 12:05:54,342 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = wsfXz4, environmentId = 1
2023-11-03 12:05:54,818 - compete.Traced

model coef_=[-2.66980549e-10  7.41467554e-10 -4.68181470e-14 -3.53841262e-11
 -6.70242724e-12 -5.49105310e-11 -1.86715183e-12 -6.79936489e-12
 -8.52782081e-11  3.03251452e-11 -2.71839114e-11 -2.01704434e-11
 -3.68070505e-11 -4.00157650e-11 -6.55436556e-12  1.74012402e-11
  3.11528789e-11  1.30438368e-11 -1.91194005e-11  8.78670053e-12
 -3.77438705e-11 -2.10495922e-12 -4.45687688e-11 -2.73493103e-11
 -4.27453836e-11 -8.08796086e-12 -7.68987651e-12 -2.81106596e-11
 -1.02913023e-12 -1.49566661e-11 -1.87970854e-11  2.09092162e-12
  4.55535783e-13 -1.67400815e-16 -1.31616072e-13 -1.99629376e-13
 -2.34919540e-14  6.96372213e-15 -1.26735428e-13 -1.14496086e-13
  8.57018449e-14  3.12957125e-14 -1.04777298e-14 -1.76976489e-14
 -1.23715274e-13 -8.80632373e-15  5.18669309e-14 -5.30049095e-14
  4.83554169e-15  3.30725031e-14 -1.92641042e-14 -3.29862006e-14
  6.10735421e-14 -7.85829735e-16 -7.40249875e-15  5.33199786e-15
  7.68765477e-14  6.78480709e-14 -8.37697967e-15 -2.14641672e-14
 -5.33999928e

2023-11-03 12:07:49,174 - compete.TracedPerInstLinearRegression - WARNING - Basic block #36 not in traced block list. 
2023-11-03 12:07:49,177 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = DllSWX, environmentId = 1
2023-11-03 12:07:52,643 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-11-03 12:07:52,646 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 1
2023-11-03 12:07:53,395 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-11-03 12:07:53,398 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 1
2023-11-03 12:07:56,083 - compete.TracedPerInstLinearRegression - WARNING - Basic block #21 not in traced block list. 
2023-11-03 12:07:56,085 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = wsfXz4, environmentId = 1
2023-11-03 12:07:56,565 - compete.Traced

...

2023-11-03 12:34:19,470 - toyDb.utils.spv.SpvInstStat - WARNING - Basic block #36 not in traced block list. 
2023-11-03 12:34:19,473 - compete.TracedLinearRegression - INFO - Occured in shaderID = DllSWX, environmentId = 1
2023-11-03 12:34:26,212 - toyDb.utils.spv.SpvInstStat - WARNING - Basic block #12 not in traced block list. 
2023-11-03 12:34:26,216 - compete.TracedLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 1
2023-11-03 12:34:27,758 - toyDb.utils.spv.SpvInstStat - WARNING - Basic block #14 not in traced block list. 
2023-11-03 12:34:27,761 - compete.TracedLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 1
2023-11-03 12:34:32,998 - toyDb.utils.spv.SpvInstStat - WARNING - Basic block #21 not in traced block list. 
2023-11-03 12:34:33,001 - compete.TracedLinearRegression - INFO - Occured in shaderID = wsfXz4, environmentId = 1
2023-11-03 12:34:34,064 - toyDb.utils.spv.SpvInstStat - WARNING - Basic block #32 not in traced block list. 

model coef_=[ 4.9456941e-07  1.8308010e-06 -1.8332277e-05  6.8138206e-10
 -6.5074346e-10 -6.1911720e-09 -2.5511326e-10  2.5295321e-11
 -3.0809133e-11 -1.2323244e-07  2.8169034e-06  0.0000000e+00
  2.0246812e-06  1.4546803e-06 -1.6378438e-06 -3.4073794e-06
 -5.3631011e-06  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00], intercept_=-5.895148206036538e-06
- Model coef: [ 4.9456941e-07  1.8308010e-06 -1.8332277e-05  6.8138206e-10
 -6.5074346e-10 -6.1911720e-09 -2.5511326e-10  2.5295321e-11
 -3.0809133e-11 -1.2323244e-07  2.8169034e-06  0.0000000e+00
  2.0246812e-06  1.4546803e-06 -1.6378438e-06 -3.4073794e-06
 -5.3631011e-06  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
- Model intercept: -5.895148206036538e-06


Evaluation: 100%|██████████| 2255/2255 [00:41<00:00, 54.11it/s]
2023-11-03 12:38:24,628 - __main__ - INFO - Running test for AllSpvCategories-withoutTrace-useMSE
Training: 100%|██████████| 9019/9019 [03:01<00:00, 49.77it/s, id=ttVfzR]


model coef_=[ 7.0574974e-06  5.2356125e-05 -6.6398195e-04 -2.0563675e-09
 -1.8571882e-09  1.7904376e-06  9.8225428e-11 -3.2741809e-11
  7.2759576e-11 -1.9072468e-06  1.3992804e-05  0.0000000e+00
  6.8777827e-06  1.6456255e-05 -1.4940327e-05 -8.1146252e-05
 -1.2099895e-04  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00], intercept_=-0.00020713376579806209
- Model coef: [ 7.0574974e-06  5.2356125e-05 -6.6398195e-04 -2.0563675e-09
 -1.8571882e-09  1.7904376e-06  9.8225428e-11 -3.2741809e-11
  7.2759576e-11 -1.9072468e-06  1.3992804e-05  0.0000000e+00
  6.8777827e-06  1.6456255e-05 -1.4940327e-05 -8.1146252e-05
 -1.2099895e-04  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
- Model intercept: -0.00020713376579806209


Evaluation: 100%|██████████| 2255/2255 [00:42<00:00, 53.60it/s]


In [8]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metrics)

{   'AllSpvCategories-withTrace-useApproxMape': {   'mae': 0.00059715175,
                                                    'mae_ge_10000fps': 1.2613889e-05,
                                                    'mae_ge_1000_le_10000fps': 0.00021419913,
                                                    'mae_ge_100_le_1000fps': 0.0018768577,
                                                    'mae_ge_10_le_100fps': 0.014059252,
                                                    'mae_le_10fps': 0.123639986,
                                                    'mape': 0.510186,
                                                    'mape_ge_10000fps': 0.46642312,
                                                    'mape_ge_1000_le_10000fps': 0.5504213,
                                                    'mape_ge_100_le_1000fps': 0.6082841,
                                                    'mape_ge_10_le_100fps': 0.74886304,
                                                    'mape_le_10f

Pack them into a table for better exploration.

In [17]:
colSets = [
    [
        'mae',
        'mape',
        'mse_sqrt'
    ],
    [
        'mae',
        'mae_ge_10000fps',
        'mae_ge_1000_le_10000fps',
        'mae_ge_100_le_1000fps',
        'mae_ge_10_le_100fps',
        'mae_le_10fps',
    ],
    [
        'mape',
        'mape_ge_10000fps',
        'mape_ge_1000_le_10000fps',
        'mape_ge_100_le_1000fps',
        'mape_ge_10_le_100fps',
        'mape_le_10fps',
    ],
    [
        'mse_sqrt',
        'mse_sqrt_ge_10000fps',
        'mse_sqrt_ge_1000_le_10000fps',
        'mse_sqrt_ge_100_le_1000fps',
        'mse_sqrt_ge_10_le_100fps',
        'mse_sqrt_le_10fps'
    ]
]

rows = [i for i in metrics.keys()]

from IPython.display import HTML, display



for cols in colSets:
    table = [['kind'] + cols]
    for row in rows:
        colContent = [row]
        
        for col in cols:
            colContent.append(metrics[row][col])
        table.append(colContent)

    tableView = tabulate.tabulate(table, headers="firstrow", tablefmt='html')
    display(HTML(tableView))

kind,mae,mape,mse_sqrt
PerInst-withTrace-useApproxMape,0.000497898,0.434064,0.00341131
PerInst-withTrace-useMSE,0.000623025,1.62107,0.00497078
PerInst-withoutTrace-useApproxMape,0.000877513,0.886048,0.00574431
PerInst-withoutTrace-useMSE,0.00115807,11.4846,0.00558509
NumInsts-withTrace-useApproxMape,0.000671673,0.53347,0.00459175
NumInsts-withTrace-useMSE,0.000675765,9.15838,0.00357452
NumInsts-withoutTrace-useApproxMape,0.000908101,0.808896,0.00576231
NumInsts-withoutTrace-useMSE,0.00122881,14.1846,0.00566499
AllSpvCategories-withTrace-useApproxMape,0.000597152,0.510186,0.00341118
AllSpvCategories-withTrace-useMSE,0.00058218,5.03007,0.00334507


kind,mae,mae_ge_10000fps,mae_ge_1000_le_10000fps,mae_ge_100_le_1000fps,mae_ge_10_le_100fps,mae_le_10fps
PerInst-withTrace-useApproxMape,0.000497898,1.15806e-05,0.000182449,0.00159235,0.0105328,0.135231
PerInst-withTrace-useMSE,0.000623025,5.36389e-05,0.000224627,0.00189269,0.0134051,0.139293
PerInst-withoutTrace-useApproxMape,0.000877513,2.67093e-05,0.000240699,0.0027397,0.0204454,0.223649
PerInst-withoutTrace-useMSE,0.00115807,0.000464859,0.00116806,0.00150954,0.0179364,0.222703
NumInsts-withTrace-useApproxMape,0.000671673,1.2376e-05,0.000229705,0.00206769,0.0151489,0.185373
NumInsts-withTrace-useMSE,0.000675765,0.000331619,0.000274392,0.00129495,0.010252,0.147145
NumInsts-withoutTrace-useApproxMape,0.000908101,2.18511e-05,0.000304275,0.00285897,0.0205883,0.223695
NumInsts-withoutTrace-useMSE,0.00122881,0.000610191,0.000894411,0.00171025,0.0191287,0.223645
AllSpvCategories-withTrace-useApproxMape,0.000597152,1.26139e-05,0.000214199,0.00187686,0.0140593,0.12364
AllSpvCategories-withTrace-useMSE,0.00058218,0.000180287,0.000242957,0.00143061,0.00997219,0.130459


kind,mape,mape_ge_10000fps,mape_ge_1000_le_10000fps,mape_ge_100_le_1000fps,mape_ge_10_le_100fps,mape_le_10fps
PerInst-withTrace-useApproxMape,0.434064,0.393226,0.483002,0.515481,0.571408,0.604509
PerInst-withTrace-useMSE,1.62107,2.25242,0.653434,0.594263,0.60533,0.622665
PerInst-withoutTrace-useApproxMape,0.886048,1.01629,0.535387,0.901936,0.985324,0.999752
PerInst-withoutTrace-useMSE,11.4846,16.9502,4.42093,0.512965,0.834347,0.995522
NumInsts-withTrace-useApproxMape,0.53347,0.480683,0.572034,0.676405,0.756973,0.828652
NumInsts-withTrace-useMSE,9.15838,14.4532,1.13989,0.390702,0.533892,0.657768
NumInsts-withoutTrace-useApproxMape,0.808896,0.808876,0.712595,0.9556,0.994185,0.999957
NumInsts-withoutTrace-useMSE,14.1846,21.6468,3.72168,0.491177,0.908655,0.999736
AllSpvCategories-withTrace-useApproxMape,0.510186,0.466423,0.550421,0.608284,0.748863,0.552694
AllSpvCategories-withTrace-useMSE,5.03007,7.77376,0.884576,0.478885,0.487679,0.583177


kind,mse_sqrt,mse_sqrt_ge_10000fps,mse_sqrt_ge_1000_le_10000fps,mse_sqrt_ge_100_le_1000fps,mse_sqrt_ge_10_le_100fps,mse_sqrt_le_10fps
PerInst-withTrace-useApproxMape,0.00341131,5.60914e-05,0.000291642,0.00246334,0.0127972,0.135231
PerInst-withTrace-useMSE,0.00497078,0.000107856,0.000433039,0.00728611,0.0232847,0.139293
PerInst-withoutTrace-useApproxMape,0.00574431,4.32859e-05,0.000324747,0.00332776,0.0238033,0.223649
PerInst-withoutTrace-useMSE,0.00558509,0.000785333,0.00145186,0.00214933,0.0216613,0.222703
NumInsts-withTrace-useApproxMape,0.00459175,3.9101e-05,0.000292726,0.00261788,0.0172816,0.185373
NumInsts-withTrace-useMSE,0.00357452,0.000339759,0.00036726,0.00192427,0.0125245,0.147145
NumInsts-withoutTrace-useApproxMape,0.00576231,3.21789e-05,0.0003831,0.00342934,0.023926,0.223695
NumInsts-withoutTrace-useMSE,0.00566499,0.000809394,0.00109301,0.00248546,0.0226191,0.223645
AllSpvCategories-withTrace-useApproxMape,0.00341118,5.1597e-05,0.000290197,0.00247544,0.0156318,0.12364
AllSpvCategories-withTrace-useMSE,0.00334507,0.000193471,0.000419065,0.00228898,0.0132279,0.130459


> TODO: add performance on training set

## 6600xt environment

In [5]:
metrics = {}

trainDataset = build_dataset('FragPerfSnapshotTracedDataset4096-6600xt-train', 'train')
testDataset = build_dataset('FragPerfSnapshotTracedDataset4096-6600xt-train', 'test')

for groupName in ['PerInst', 'NumInsts', 'AllSpvCategories']:
  for enableTrace in [True, False]:
    for useApproxMape in [True, False]:
      fullName = f"{groupName}-{'withTrace' if enableTrace else 'withoutTrace'}-{'useApproxMape' if useApproxMape else 'useMSE'}"
      logger.info(f"Running test for {fullName}")

      with logging_redirect_tqdm():
        metrics[fullName] = do_compute(**{
          'groupName': groupName,
          'enableTrace': enableTrace,
          'useApproxMape': useApproxMape,
          'trainDataset': trainDataset,
          'testDataset': testDataset
        })

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metrics)

colSets = [
    [
        'mae',
        'mape',
        'mse_sqrt'
    ],
    [
        'mae',
        'mae_ge_10000fps',
        'mae_ge_1000_le_10000fps',
        'mae_ge_100_le_1000fps',
        'mae_ge_10_le_100fps',
        'mae_le_10fps',
    ],
    [
        'mape',
        'mape_ge_10000fps',
        'mape_ge_1000_le_10000fps',
        'mape_ge_100_le_1000fps',
        'mape_ge_10_le_100fps',
        'mape_le_10fps',
    ],
    [
        'mse_sqrt',
        'mse_sqrt_ge_10000fps',
        'mse_sqrt_ge_1000_le_10000fps',
        'mse_sqrt_ge_100_le_1000fps',
        'mse_sqrt_ge_10_le_100fps',
        'mse_sqrt_le_10fps'
    ]
]

rows = [i for i in metrics.keys()]

from IPython.display import HTML, display

for cols in colSets:
    table = [['kind'] + cols]
    for row in rows:
        colContent = [row]
        
        for col in cols:
            colContent.append(metrics[row][col])
        table.append(colContent)

    tableView = tabulate.tabulate(table, headers="firstrow", tablefmt='html')
    display(HTML(tableView))

2023-12-08 14:46:32,209 - __main__ - INFO - Running test for PerInst-withTrace-useApproxMape
2023-12-08 14:46:32,225 - compete.TracedPerInstLinearRegression - INFO - Collected 712 inst names from grammar
2023-12-08 14:46:42,169 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-08 14:46:42,172 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = XtVfWD, environmentId = 2
2023-12-08 14:46:50,065 - compete.TracedPerInstLinearRegression - WARNING - Basic block #27 not in traced block list. 
2023-12-08 14:46:50,071 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = Wds3zf, environmentId = 2
2023-12-08 14:46:59,471 - compete.TracedPerInstLinearRegression - WARNING - Basic block #15 not in traced block list. 
2023-12-08 14:46:59,474 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = lsffRX, environmentId = 2
2023-12-08 14:47:01,633 - compete.TracedPerInstLinearRegression - WARNING - 

model coef_=[-1.78016557e-09 -3.05889047e-09 -1.31712512e-13 -1.92289101e-10
 -6.26406704e-11  1.85640253e-10 -1.71585727e-13  2.48047329e-12
 -3.93265726e-10 -4.49569465e-10 -1.77563936e-10  3.03611219e-10
 -2.78588597e-10  2.12055554e-10 -1.16704188e-10  2.03565387e-10
  1.52152235e-10  3.61437026e-11 -3.53991871e-11 -5.54416200e-11
 -5.69928583e-11 -3.15920922e-12 -1.37226217e-10 -7.18520868e-11
  1.25302477e-11 -3.05640084e-12  1.27210742e-10  5.77958306e-11
 -3.40226840e-11  7.66134031e-11 -4.08924769e-11  1.04633087e-10
 -7.96602576e-11 -2.22458120e-13 -8.54397282e-13  5.92508247e-13
 -2.80892930e-14  4.25038477e-13 -1.21314569e-12 -3.81847982e-13
  6.31068114e-13  8.97640903e-13 -4.36911358e-13  2.55067668e-13
 -1.75504142e-13  3.14170131e-13 -1.95404023e-13 -3.83415522e-13
  3.62349040e-14 -4.51746279e-13  7.80746995e-14 -4.90963173e-13
 -1.89371088e-13  4.16697926e-14  9.10946665e-14  3.00170479e-13
  1.78055487e-13 -5.52813004e-14 -1.34398569e-13  6.37271919e-13
  5.88860558e

2023-12-08 14:48:21,970 - compete.TracedPerInstLinearRegression - WARNING - Basic block #32 not in traced block list. 
2023-12-08 14:48:21,974 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = WtlGDB, environmentId = 2
2023-12-08 14:48:23,454 - compete.TracedPerInstLinearRegression - WARNING - Basic block #21 not in traced block list. 
2023-12-08 14:48:23,458 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = wsfXz4, environmentId = 2
2023-12-08 14:48:28,736 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-08 14:48:28,740 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 2
2023-12-08 14:48:29,341 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-12-08 14:48:29,345 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 2
2023-12-08 14:48:32,818 - compete.Traced

model coef_=[-4.39517207e-11  1.40676326e-09 -1.84759868e-13  8.60769928e-11
  8.27557883e-11 -2.69851419e-10 -1.09114378e-12 -7.46272293e-13
  2.06851376e-11  1.04105309e-11  1.90951893e-10 -1.84667594e-11
 -1.63330946e-10 -6.00225564e-11  3.54611479e-11  2.65379836e-11
 -2.13864565e-10  1.14220742e-11  3.98792457e-11  7.94409191e-11
  2.76879682e-11  1.15086343e-11  8.49082887e-11  2.71727649e-12
  4.46326101e-11  4.24889984e-11 -6.98409663e-11 -3.61102259e-11
 -1.99728810e-11 -6.24239063e-11  4.05949614e-11 -6.77319242e-11
  2.14862555e-11 -1.19758587e-13 -8.86050131e-14  7.96371432e-14
  5.48467521e-14  5.19142997e-13 -2.33115393e-13  1.59656359e-13
 -4.38073650e-14 -3.27320636e-15  1.67566482e-13 -1.19076840e-14
 -2.36099443e-13 -1.81486770e-13 -7.96186034e-14 -1.40376426e-13
 -4.96599289e-14 -2.81762461e-14 -5.24146698e-14 -1.66091099e-14
  8.02370323e-14 -4.13745427e-13  2.39510668e-13  4.79928597e-14
 -1.10439435e-13 -3.82567242e-14 -2.50468049e-13  1.43472040e-13
 -3.47742668e

2023-12-08 14:50:32,763 - compete.TracedPerInstLinearRegression - WARNING - Basic block #32 not in traced block list. 
2023-12-08 14:50:32,767 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = WtlGDB, environmentId = 2
2023-12-08 14:50:34,266 - compete.TracedPerInstLinearRegression - WARNING - Basic block #21 not in traced block list. 
2023-12-08 14:50:34,269 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = wsfXz4, environmentId = 2
2023-12-08 14:50:39,507 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-08 14:50:39,511 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 2
2023-12-08 14:50:40,122 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-12-08 14:50:40,126 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 2
2023-12-08 14:50:43,598 - compete.Traced

...

{   'AllSpvCategories-withTrace-useApproxMape': {   'mae': 0.000458649,
                                                    'mae_ge_10000fps': 1.907887e-05,
                                                    'mae_ge_1000_le_10000fps': 0.0002338826,
                                                    'mae_ge_100_le_1000fps': 0.0013325952,
                                                    'mae_ge_10_le_100fps': 0.0109865,
                                                    'mae_le_10fps': nan,
                                                    'mape': 0.60758895,
                                                    'mape_ge_10000fps': 0.7225258,
                                                    'mape_ge_1000_le_10000fps': 0.530791,
                                                    'mape_ge_100_le_1000fps': 0.47655973,
                                                    'mape_ge_10_le_100fps': 0.5913433,
                                                    'mape_le_10fps': nan,
    

kind,mae,mape,mse_sqrt
PerInst-withTrace-useApproxMape,0.00039821,0.584053,0.00153308
PerInst-withTrace-useMSE,0.000447405,2.38493,0.00160926
PerInst-withoutTrace-useApproxMape,0.000726992,0.924843,0.00262737
PerInst-withoutTrace-useMSE,0.00090248,4.77607,0.00240553
NumInsts-withTrace-useApproxMape,0.000501574,0.670366,0.00167976
NumInsts-withTrace-useMSE,0.000588381,5.82278,0.00164462
NumInsts-withoutTrace-useApproxMape,0.000789076,0.882322,0.00269491
NumInsts-withoutTrace-useMSE,0.000996859,6.57787,0.00252296
AllSpvCategories-withTrace-useApproxMape,0.000458649,0.607589,0.00171939
AllSpvCategories-withTrace-useMSE,0.000499146,3.22651,0.00185354


kind,mae,mae_ge_10000fps,mae_ge_1000_le_10000fps,mae_ge_100_le_1000fps,mae_ge_10_le_100fps,mae_le_10fps
PerInst-withTrace-useApproxMape,0.00039821,2.08603e-05,0.000219976,0.00120584,0.0083276,nan
PerInst-withTrace-useMSE,0.000447405,0.000160078,0.000161029,0.00118843,0.0093148,nan
PerInst-withoutTrace-useApproxMape,0.000726992,5.39515e-05,0.000216518,0.00233118,0.0184562,nan
PerInst-withoutTrace-useMSE,0.00090248,0.000319429,0.000817236,0.00142825,0.0163018,nan
NumInsts-withTrace-useApproxMape,0.000501574,1.97043e-05,0.000256839,0.00158072,0.0105496,nan
NumInsts-withTrace-useMSE,0.000588381,0.000421135,0.000225166,0.0011387,0.00998331,nan
NumInsts-withoutTrace-useApproxMape,0.000789076,4.1199e-05,0.000281393,0.0025748,0.0187388,nan
NumInsts-withoutTrace-useMSE,0.000996859,0.0004703,0.000794071,0.00156572,0.0174009,nan
AllSpvCategories-withTrace-useApproxMape,0.000458649,1.90789e-05,0.000233883,0.0013326,0.0109865,nan
AllSpvCategories-withTrace-useMSE,0.000499146,0.000225291,0.000185116,0.00119732,0.0102293,nan


kind,mape,mape_ge_10000fps,mape_ge_1000_le_10000fps,mape_ge_100_le_1000fps,mape_ge_10_le_100fps,mape_le_10fps
PerInst-withTrace-useApproxMape,0.584053,0.70665,0.517112,0.416534,0.467132,nan
PerInst-withTrace-useMSE,2.38493,4.84536,0.452076,0.402734,0.528306,nan
PerInst-withoutTrace-useApproxMape,0.924843,1.3423,0.503242,0.78418,0.969379,nan
PerInst-withoutTrace-useMSE,4.77607,8.11613,2.84397,0.536763,0.834578,nan
NumInsts-withTrace-useApproxMape,0.670366,0.807897,0.561297,0.560187,0.588814,nan
NumInsts-withTrace-useMSE,5.82278,12.0653,1.12107,0.353726,0.543063,nan
NumInsts-withoutTrace-useApproxMape,0.882322,1.14244,0.573842,0.89859,0.987039,nan
NumInsts-withoutTrace-useMSE,6.57787,11.7944,3.32967,0.472195,0.907535,nan
AllSpvCategories-withTrace-useApproxMape,0.607589,0.722526,0.530791,0.47656,0.591343,nan
AllSpvCategories-withTrace-useMSE,3.22651,6.58984,0.633842,0.41197,0.518041,nan


kind,mse_sqrt,mse_sqrt_ge_10000fps,mse_sqrt_ge_1000_le_10000fps,mse_sqrt_ge_100_le_1000fps,mse_sqrt_ge_10_le_100fps,mse_sqrt_le_10fps
PerInst-withTrace-useApproxMape,0.00153308,4.81662e-05,0.000284666,0.00247412,0.0100018,nan
PerInst-withTrace-useMSE,0.00160926,0.000215116,0.000249559,0.00200133,0.011927,nan
PerInst-withoutTrace-useApproxMape,0.00262737,8.16096e-05,0.000290606,0.0029236,0.0202677,nan
PerInst-withoutTrace-useMSE,0.00240553,0.00057934,0.00109233,0.00199584,0.0184158,nan
NumInsts-withTrace-useApproxMape,0.00167976,3.11872e-05,0.000323772,0.0020356,0.0125771,nan
NumInsts-withTrace-useMSE,0.00164462,0.00042221,0.000261283,0.00170168,0.0126317,nan
NumInsts-withoutTrace-useApproxMape,0.00269491,6.29255e-05,0.000373991,0.00313108,0.0205214,nan
NumInsts-withoutTrace-useMSE,0.00252296,0.000632349,0.000985194,0.00229359,0.0192717,nan
AllSpvCategories-withTrace-useApproxMape,0.00171939,3.55567e-05,0.00030019,0.00195438,0.0131348,nan
AllSpvCategories-withTrace-useMSE,0.00185354,0.00022914,0.000250656,0.0021397,0.0140838,nan


## uhd630

In [5]:
metrics = {}

trainDataset = build_dataset('FragPerfSnapshotTracedDataset4096-uhd630', 'train')
testDataset = build_dataset('FragPerfSnapshotTracedDataset4096-uhd630', 'test')

for groupName in ['PerInst', 'NumInsts', 'AllSpvCategories']:
  for enableTrace in [True, False]:
    for useApproxMape in [True, False]:
      fullName = f"{groupName}-{'withTrace' if enableTrace else 'withoutTrace'}-{'useApproxMape' if useApproxMape else 'useMSE'}"
      logger.info(f"Running test for {fullName}")

      with logging_redirect_tqdm():
        metrics[fullName] = do_compute(**{
          'groupName': groupName,
          'enableTrace': enableTrace,
          'useApproxMape': useApproxMape,
          'trainDataset': trainDataset,
          'testDataset': testDataset
        })

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metrics)

colSets = [
    [
        'mae',
        'mape',
        'mse_sqrt'
    ],
    [
        'mae',
        'mae_ge_10000fps',
        'mae_ge_1000_le_10000fps',
        'mae_ge_100_le_1000fps',
        'mae_ge_10_le_100fps',
        'mae_le_10fps',
    ],
    [
        'mape',
        'mape_ge_10000fps',
        'mape_ge_1000_le_10000fps',
        'mape_ge_100_le_1000fps',
        'mape_ge_10_le_100fps',
        'mape_le_10fps',
    ],
    [
        'mse_sqrt',
        'mse_sqrt_ge_10000fps',
        'mse_sqrt_ge_1000_le_10000fps',
        'mse_sqrt_ge_100_le_1000fps',
        'mse_sqrt_ge_10_le_100fps',
        'mse_sqrt_le_10fps'
    ]
]

rows = [i for i in metrics.keys()]

from IPython.display import HTML, display

for cols in colSets:
    table = [['kind'] + cols]
    for row in rows:
        colContent = [row]
        
        for col in cols:
            colContent.append(metrics[row][col])
        table.append(colContent)

    tableView = tabulate.tabulate(table, headers="firstrow", tablefmt='html')
    display(HTML(tableView))

2023-12-11 10:01:52,288 - __main__ - INFO - Running test for PerInst-withTrace-useApproxMape
2023-12-11 10:01:52,300 - compete.TracedPerInstLinearRegression - INFO - Collected 712 inst names from grammar
2023-12-11 10:02:08,948 - compete.TracedPerInstLinearRegression - WARNING - Basic block #15 not in traced block list. 
2023-12-11 10:02:08,952 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = lsffRX, environmentId = 3
2023-12-11 10:02:31,876 - compete.TracedPerInstLinearRegression - WARNING - Basic block #25 not in traced block list. 
2023-12-11 10:02:31,881 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = 3tKXzc, environmentId = 3
2023-12-11 10:02:39,046 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-11 10:02:39,050 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = XtVfWD, environmentId = 3
2023-12-11 10:03:01,114 - compete.TracedPerInstLinearRegression - WARNING - 

model coef_=[-3.40643447e-09 -1.83530435e-08 -1.39144078e-12 -9.40382217e-10
 -1.71503811e-09  5.52021595e-10 -1.32591368e-11  1.13911582e-10
  4.70657457e-10  3.28810035e-11 -6.24602925e-10 -3.57350607e-11
 -2.72595974e-11  8.62069166e-11 -8.23016169e-11 -1.49342955e-10
  4.50659822e-11  1.02197570e-10 -4.53185232e-12 -2.57781990e-11
  6.33548075e-11 -1.12524963e-12  2.06265317e-11 -4.30446026e-11
 -2.20218843e-11  1.16228112e-11  1.59396194e-11 -6.34969291e-12
 -1.24210043e-11  5.79496173e-12  1.04979341e-11 -1.32761918e-11
  2.33097916e-12 -1.55139226e-12  1.39100727e-11 -3.59393938e-12
  1.72083875e-12  5.69813641e-12 -2.61575483e-13 -7.17721022e-13
  3.31890418e-12 -1.86325955e-12 -2.09787049e-13 -1.03373213e-12
  5.11439155e-12  7.73440340e-12 -1.07797798e-12 -1.51574413e-12
 -4.37455280e-12  1.03710859e-11 -2.86621595e-12  7.49475135e-13
  3.57836222e-12  4.99627423e-12  4.94264352e-13 -4.09436374e-13
 -2.85091395e-12 -2.88456758e-13  1.43842056e-12  2.64217814e-12
  4.51458315e

2023-12-11 10:03:17,364 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-12-11 10:03:17,368 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 3
2023-12-11 10:03:18,984 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-11 10:03:18,989 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 3
Evaluation: 100%|██████████| 1906/1906 [00:16<00:00, 115.09it/s]
c:\Projects\NGPP\vkPredict\notebooks\..\misc\metric.py:39: RuntimeWarning: Mean of empty slice.
  ((candRealTime - candPredTime)**2).mean(axis=0)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis

model coef_=[-2.03274536e+01  4.42343597e+01  5.54798171e-03  2.58616567e+00
 -7.33367145e-01 -5.32656848e-01 -1.12102173e-01  1.13140456e-01
  2.90229209e-02  9.32784155e-02  4.26849633e-01  8.41103494e-02
 -8.55803266e-02  4.60801274e-02 -8.87639076e-03 -1.91039219e-02
 -3.79274637e-02 -1.73526183e-02  7.93094337e-02 -1.84135474e-02
  6.82325736e-02 -7.81866163e-03  2.64435560e-02  2.22232193e-02
  3.52927707e-02 -3.83032300e-03 -6.45689666e-04  6.01963624e-02
 -3.26273665e-02  6.67525828e-03  3.09621096e-02  3.79759446e-03
  1.85476504e-02 -2.30564345e-02 -7.47795030e-03 -5.29072434e-03
 -1.14778578e-02  4.94508259e-03  5.23945689e-03 -1.31389312e-02
 -1.01505965e-03 -7.79300928e-04  1.32177249e-02 -6.19965792e-03
  1.06424093e-03  4.81620431e-04  1.01766735e-02 -3.71330976e-03
  9.99301672e-04  2.36469507e-03  8.58671963e-03  5.71787357e-04
 -2.21014023e-04  5.35279512e-04  1.03777647e-03 -4.45961952e-04
 -1.26707554e-03 -2.14523077e-03  9.90629196e-05  1.59552693e-03
  5.79401851e

2023-12-11 10:04:52,141 - compete.TracedPerInstLinearRegression - WARNING - Basic block #14 not in traced block list. 
2023-12-11 10:04:52,145 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = ttdczS, environmentId = 3
2023-12-11 10:04:53,765 - compete.TracedPerInstLinearRegression - WARNING - Basic block #12 not in traced block list. 
2023-12-11 10:04:53,768 - compete.TracedPerInstLinearRegression - INFO - Occured in shaderID = td2cDy, environmentId = 3
Evaluation: 100%|██████████| 1906/1906 [00:16<00:00, 114.88it/s]
c:\Projects\NGPP\vkPredict\notebooks\..\misc\metric.py:39: RuntimeWarning: Mean of empty slice.
  ((candRealTime - candPredTime)**2).mean(axis=0)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis

...

{   'AllSpvCategories-withTrace-useApproxMape': {   'mae': 0.0013831585,
                                                    'mae_ge_10000fps': nan,
                                                    'mae_ge_1000_le_10000fps': 0.00010056293,
                                                    'mae_ge_100_le_1000fps': 0.001205691,
                                                    'mae_ge_10_le_100fps': 0.008286689,
                                                    'mae_le_10fps': 0.02977727,
                                                    'mape': 0.2881483,
                                                    'mape_ge_10000fps': nan,
                                                    'mape_ge_1000_le_10000fps': 0.25558734,
                                                    'mape_ge_100_le_1000fps': 0.34422293,
                                                    'mape_ge_10_le_100fps': 0.30229592,
                                                    'mape_le_10fps': 0.2762271,
 

kind,mae,mape,mse_sqrt
PerInst-withTrace-useApproxMape,0.00126989,0.276679,0.00453698
PerInst-withTrace-useMSE,2.94951e+06,2.58433e+09,7.8556e+06
PerInst-withoutTrace-useApproxMape,0.00335592,0.707854,0.0101167
PerInst-withoutTrace-useMSE,3.83284e+06,5.77308e+09,6.07749e+06
NumInsts-withTrace-useApproxMape,0.0015826,0.33283,0.00556822
NumInsts-withTrace-useMSE,1.30567e+06,1.59634e+09,1.97082e+06
NumInsts-withoutTrace-useApproxMape,0.00362297,0.699704,0.0104626
NumInsts-withoutTrace-useMSE,1.37993e+06,1.66849e+09,1.7348e+06
AllSpvCategories-withTrace-useApproxMape,0.00138316,0.288148,0.0047736
AllSpvCategories-withTrace-useMSE,2.15741e+06,2.74216e+09,4.5852e+06


kind,mae,mae_ge_10000fps,mae_ge_1000_le_10000fps,mae_ge_100_le_1000fps,mae_ge_10_le_100fps,mae_le_10fps
PerInst-withTrace-useApproxMape,0.00126989,nan,9.71705e-05,0.00114693,0.00759057,0.0147688
PerInst-withTrace-useMSE,2.94951e+06,nan,1.04097e+06,2.85136e+06,1.27223e+07,4.94093e+07
PerInst-withoutTrace-useApproxMape,0.00335592,nan,0.000272477,0.00213115,0.021848,0.105339
PerInst-withoutTrace-useMSE,3.83284e+06,nan,2.97896e+06,4.52293e+06,6.28329e+06,1.25353e+07
NumInsts-withTrace-useApproxMape,0.0015826,nan,0.000120666,0.00135054,0.00927676,0.0612138
NumInsts-withTrace-useMSE,1.30567e+06,nan,657210,1.41812e+06,4.27729e+06,1.12032e+07
NumInsts-withoutTrace-useApproxMape,0.00362297,nan,0.000247288,0.00265044,0.0229281,0.106577
NumInsts-withoutTrace-useMSE,1.37993e+06,nan,896265,1.95297e+06,2.30137e+06,2.21812e+06
AllSpvCategories-withTrace-useApproxMape,0.00138316,nan,0.000100563,0.00120569,0.00828669,0.0297773
AllSpvCategories-withTrace-useMSE,2.15741e+06,nan,1.13273e+06,1.90283e+06,8.03364e+06,1.9741e+07


kind,mape,mape_ge_10000fps,mape_ge_1000_le_10000fps,mape_ge_100_le_1000fps,mape_ge_10_le_100fps,mape_le_10fps
PerInst-withTrace-useApproxMape,0.276679,nan,0.249544,0.327418,0.278558,0.13909
PerInst-withTrace-useMSE,2.58433e+09,nan,3.89037e+09,8.66365e+08,5.68049e+08,4.53702e+08
PerInst-withoutTrace-useApproxMape,0.707854,nan,0.799809,0.477235,0.866101,0.976299
PerInst-withoutTrace-useMSE,5.77308e+09,nan,8.88158e+09,1.92392e+09,3.14742e+08,1.15856e+08
NumInsts-withTrace-useApproxMape,0.33283,nan,0.303785,0.38503,0.337096,0.56638
NumInsts-withTrace-useMSE,1.59634e+09,nan,2.46111e+09,4.8059e+08,2.01654e+08,1.04048e+08
NumInsts-withoutTrace-useApproxMape,0.699704,nan,0.711705,0.595111,0.924589,0.987735
NumInsts-withoutTrace-useMSE,1.66849e+09,nan,2.42949e+09,8.01213e+08,1.26592e+08,2.05868e+07
AllSpvCategories-withTrace-useApproxMape,0.288148,nan,0.255587,0.344223,0.302296,0.276227
AllSpvCategories-withTrace-useMSE,2.74216e+09,nan,4.32859e+09,6.31599e+08,3.58708e+08,1.81276e+08


kind,mse_sqrt,mse_sqrt_ge_10000fps,mse_sqrt_ge_1000_le_10000fps,mse_sqrt_ge_100_le_1000fps,mse_sqrt_ge_10_le_100fps,mse_sqrt_le_10fps
PerInst-withTrace-useApproxMape,0.00453698,nan,0.000228842,0.00219204,0.0129413,0.0205599
PerInst-withTrace-useMSE,7.8556e+06,nan,1.10769e+06,4.57663e+06,2.14383e+07,5.70725e+07
PerInst-withoutTrace-useApproxMape,0.0101167,nan,0.000445915,0.00300925,0.0280947,0.105351
PerInst-withoutTrace-useMSE,6.07749e+06,nan,4.84511e+06,6.72301e+06,9.10668e+06,1.27464e+07
NumInsts-withTrace-useApproxMape,0.00556822,nan,0.000262342,0.00227816,0.0151048,0.0616405
NumInsts-withTrace-useMSE,1.97082e+06,nan,661710,1.60528e+06,4.92892e+06,1.12722e+07
NumInsts-withoutTrace-useApproxMape,0.0104626,nan,0.000392442,0.00359045,0.02899,0.106593
NumInsts-withoutTrace-useMSE,1.7348e+06,nan,1.18373e+06,2.20873e+06,2.48656e+06,2.22362e+06
AllSpvCategories-withTrace-useApproxMape,0.0047736,nan,0.000215345,0.00210658,0.0135626,0.0298002
AllSpvCategories-withTrace-useMSE,4.5852e+06,nan,1.16118e+06,3.03581e+06,1.23099e+07,2.27915e+07


## 3060-optim20000

In [5]:
metrics = {}

trainDataset = build_dataset('FragPerfSnapshotTracedDataset4096-3060-Optim20000', 'train')
testDataset = build_dataset('FragPerfSnapshotTracedDataset4096-3060-Optim20000', 'test')

for groupName in ['PerInst', 'NumInsts', 'AllSpvCategories']:
  for enableTrace in [True, False]:
    for useApproxMape in [True, False]:
      fullName = f"{groupName}-{'withTrace' if enableTrace else 'withoutTrace'}-{'useApproxMape' if useApproxMape else 'useMSE'}"
      logger.info(f"Running test for {fullName}")

      with logging_redirect_tqdm():
        metrics[fullName] = do_compute(**{
          'groupName': groupName,
          'enableTrace': enableTrace,
          'useApproxMape': useApproxMape,
          'trainDataset': trainDataset,
          'testDataset': testDataset
        })

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metrics)

colSets = [
    [
        'mae',
        'mape',
        'mse_sqrt'
    ],
    [
        'mae',
        'mae_ge_10000fps',
        'mae_ge_1000_le_10000fps',
        'mae_ge_100_le_1000fps',
        'mae_ge_10_le_100fps',
        'mae_le_10fps',
    ],
    [
        'mape',
        'mape_ge_10000fps',
        'mape_ge_1000_le_10000fps',
        'mape_ge_100_le_1000fps',
        'mape_ge_10_le_100fps',
        'mape_le_10fps',
    ],
    [
        'mse_sqrt',
        'mse_sqrt_ge_10000fps',
        'mse_sqrt_ge_1000_le_10000fps',
        'mse_sqrt_ge_100_le_1000fps',
        'mse_sqrt_ge_10_le_100fps',
        'mse_sqrt_le_10fps'
    ]
]

rows = [i for i in metrics.keys()]

from IPython.display import HTML, display

for cols in colSets:
    table = [['kind'] + cols]
    for row in rows:
        colContent = [row]
        
        for col in cols:
            colContent.append(metrics[row][col])
        table.append(colContent)

    tableView = tabulate.tabulate(table, headers="firstrow", tablefmt='html')
    display(HTML(tableView))

2023-12-14 00:36:44,182 - __main__ - INFO - Running test for PerInst-withTrace-useApproxMape
2023-12-14 00:36:44,204 - compete.TracedPerInstLinearRegression - INFO - Collected 712 inst names from grammar
Training: 100%|██████████| 8326/8326 [01:09<00:00, 119.33it/s, id=MsjyWR]


model coef_=[ 2.15737869e-13 -7.29126670e-09  6.93536157e-13 -1.08192155e-09
 -1.28377298e-09  1.91756144e-09  1.33990779e-12  6.33850090e-12
  6.56201760e-10  5.74089387e-10 -4.53692250e-10 -1.73453502e-10
  1.04067276e-10  9.98413366e-11 -2.06667433e-10 -8.17010556e-11
 -2.75637776e-11  2.84507903e-11  4.26117787e-11  6.63768901e-11
 -7.64874275e-11  1.20359729e-11  3.81152471e-11  1.01603431e-11
  2.75894672e-11 -1.22216022e-11  2.36624383e-11 -6.19448329e-12
  1.42721650e-11 -2.11822452e-11  6.16690683e-12  4.51286838e-12
 -3.01088538e-12  1.42610121e-12  2.34760562e-12 -3.77582904e-12
 -2.36306114e-12  5.36308411e-13  1.15130041e-12  2.31293196e-12
  9.66350264e-13 -6.24079347e-13  1.67192996e-12  1.96637324e-12
  2.52428113e-12  3.72364466e-13  5.12348150e-12  7.02857911e-13
 -8.00869787e-14  6.22659910e-13  1.08782601e-12  5.89086072e-13
 -3.90520949e-13 -6.38836206e-13 -5.35547301e-13  2.93418068e-13
  3.46663670e-13  3.31110139e-13 -2.46785231e-13  3.97428618e-13
  4.54974600e

Evaluation: 100%|██████████| 2110/2110 [00:13<00:00, 153.06it/s]
2023-12-14 00:38:08,422 - __main__ - INFO - Running test for PerInst-withTrace-useMSE
2023-12-14 00:38:08,425 - compete.TracedPerInstLinearRegression - INFO - Collected 712 inst names from grammar
Training: 100%|██████████| 8326/8326 [01:10<00:00, 118.83it/s, id=MsjyWR]


model coef_=[ 1.22264441e-17 -1.85953850e-10  2.79950000e-12 -5.35891817e-11
  2.29267161e-11  8.04086381e-11  6.22249214e-13  1.36375828e-12
  1.10914966e-11  9.48822878e-12 -1.41212031e-12 -6.91431374e-12
 -2.57111121e-11 -1.28942976e-11 -1.33315511e-11  1.31392310e-11
 -1.33859341e-12 -3.03365492e-11  2.47674407e-12 -1.83896717e-11
  5.10841499e-12  5.99083153e-12 -1.32193909e-11 -8.47264186e-12
 -6.88082317e-12 -1.34963655e-13 -3.96409685e-13  1.31167022e-11
 -2.33648387e-13 -3.45412826e-12 -9.76873747e-13  7.26344332e-12
 -3.48798941e-12  5.45963014e-14 -7.06691650e-14 -1.11889231e-13
 -7.99603439e-14  1.46218541e-13  4.39496529e-14 -7.78370424e-15
 -7.42288175e-15  4.59745089e-15  1.06826440e-13  3.36989551e-14
 -8.75699253e-15 -1.01074504e-13 -2.88124559e-14  6.24023402e-15
  7.01023441e-14 -2.53957960e-14  3.45929882e-14 -1.04953264e-13
  8.61723887e-16 -4.47557573e-14  7.18500780e-15  1.04300249e-15
 -9.14567901e-15 -4.15327495e-14  3.18456199e-14  2.92913480e-14
 -3.52521831e

Evaluation: 100%|██████████| 2110/2110 [00:13<00:00, 151.30it/s]
2023-12-14 00:39:33,108 - __main__ - INFO - Running test for PerInst-withoutTrace-useApproxMape
2023-12-14 00:39:33,119 - compete.TracedPerInstLinearRegression - INFO - Collected 712 inst names from grammar
Training: 100%|██████████| 8326/8326 [01:04<00:00, 128.16it/s, id=MsjyWR]


...

{   'AllSpvCategories-withTrace-useApproxMape': {   'mae': 0.00044540782,
                                                    'mae_ge_10000fps': 1.4281062e-05,
                                                    'mae_ge_1000_le_10000fps': 0.00016645058,
                                                    'mae_ge_100_le_1000fps': 0.0014546764,
                                                    'mae_ge_10_le_100fps': 0.011806437,
                                                    'mae_le_10fps': 0.17343424,
                                                    'mape': 0.53536344,
                                                    'mape_ge_10000fps': 0.57722735,
                                                    'mape_ge_1000_le_10000fps': 0.43533173,
                                                    'mape_ge_100_le_1000fps': 0.48888904,
                                                    'mape_ge_10_le_100fps': 0.5868386,
                                                    'mape_le_1

kind,mae,mape,mse_sqrt
PerInst-withTrace-useApproxMape,0.000382865,0.674493,0.00354434
PerInst-withTrace-useMSE,0.000446272,1.03261,0.0045043
PerInst-withoutTrace-useApproxMape,0.00075097,0.71968,0.00576843
PerInst-withoutTrace-useMSE,0.00102354,9.8747,0.00567567
NumInsts-withTrace-useApproxMape,0.000477726,0.640753,0.00361754
NumInsts-withTrace-useMSE,0.000611736,9.27489,0.0033172
NumInsts-withoutTrace-useApproxMape,0.000775144,0.6881,0.00578434
NumInsts-withoutTrace-useMSE,0.00105124,11.8143,0.00571178
AllSpvCategories-withTrace-useApproxMape,0.000445408,0.535363,0.00416913
AllSpvCategories-withTrace-useMSE,0.000443158,3.68546,0.00333668


kind,mae,mae_ge_10000fps,mae_ge_1000_le_10000fps,mae_ge_100_le_1000fps,mae_ge_10_le_100fps,mae_le_10fps
PerInst-withTrace-useApproxMape,0.000382865,1.83206e-05,0.000145976,0.0013314,0.00937396,0.142397
PerInst-withTrace-useMSE,0.000446272,3.08992e-05,0.000190838,0.00122939,0.0130782,0.1572
PerInst-withoutTrace-useApproxMape,0.00075097,1.97546e-05,0.000245398,0.0026457,0.0215851,0.223701
PerInst-withoutTrace-useMSE,0.00102354,0.000391188,0.000886196,0.001839,0.0196262,0.223039
NumInsts-withTrace-useApproxMape,0.000477726,1.61208e-05,0.000192568,0.00164019,0.0133157,0.141407
NumInsts-withTrace-useMSE,0.000611736,0.000314895,0.000208634,0.00129203,0.0118316,0.129681
NumInsts-withoutTrace-useApproxMape,0.000775144,1.80142e-05,0.000292744,0.00275963,0.0217353,0.223746
NumInsts-withoutTrace-useMSE,0.00105124,0.00047322,0.000720602,0.00183322,0.0205836,0.223617
AllSpvCategories-withTrace-useApproxMape,0.000445408,1.42811e-05,0.000166451,0.00145468,0.0118064,0.173434
AllSpvCategories-withTrace-useMSE,0.000443158,0.000122717,0.000175665,0.00117549,0.00991086,0.133473


kind,mape,mape_ge_10000fps,mape_ge_1000_le_10000fps,mape_ge_100_le_1000fps,mape_ge_10_le_100fps,mape_le_10fps
PerInst-withTrace-useApproxMape,0.674493,0.813009,0.408671,0.436655,0.446411,0.636391
PerInst-withTrace-useMSE,1.03261,1.30883,0.551303,0.463538,0.528772,0.702545
PerInst-withoutTrace-useApproxMape,0.71968,0.733599,0.577921,0.899461,0.986719,0.999748
PerInst-withoutTrace-useMSE,9.8747,13.8738,3.49982,0.621823,0.884391,0.996791
NumInsts-withTrace-useApproxMape,0.640753,0.704189,0.504814,0.547822,0.648519,0.631965
NumInsts-withTrace-useMSE,9.27489,13.8975,0.932534,0.39795,0.581901,0.579561
NumInsts-withoutTrace-useApproxMape,0.6881,0.632162,0.701697,0.952506,0.99455,0.999948
NumInsts-withoutTrace-useMSE,11.8143,17.062,2.97081,0.554696,0.933326,0.999373
AllSpvCategories-withTrace-useApproxMape,0.535363,0.577227,0.435332,0.488889,0.586839,0.7751
AllSpvCategories-withTrace-useMSE,3.68546,5.39762,0.589255,0.412584,0.44464,0.596509


kind,mse_sqrt,mse_sqrt_ge_10000fps,mse_sqrt_ge_1000_le_10000fps,mse_sqrt_ge_100_le_1000fps,mse_sqrt_ge_10_le_100fps,mse_sqrt_le_10fps
PerInst-withTrace-useApproxMape,0.00354434,0.000280781,0.000267542,0.00248386,0.0129036,0.142397
PerInst-withTrace-useMSE,0.0045043,0.000374319,0.000380616,0.00184692,0.024675,0.1572
PerInst-withoutTrace-useApproxMape,0.00576843,3.94271e-05,0.000322907,0.00324195,0.0251335,0.223701
PerInst-withoutTrace-useMSE,0.00567567,0.000664448,0.00129082,0.00260829,0.0233335,0.223039
NumInsts-withTrace-useApproxMape,0.00361754,0.000185021,0.000267433,0.00216799,0.0151773,0.141407
NumInsts-withTrace-useMSE,0.0033172,0.000378687,0.000286253,0.00184634,0.0138587,0.129681
NumInsts-withoutTrace-useApproxMape,0.00578434,2.88736e-05,0.000368685,0.00333707,0.0252817,0.223746
NumInsts-withoutTrace-useMSE,0.00571178,0.000603974,0.000942492,0.00260903,0.0241523,0.223617
AllSpvCategories-withTrace-useApproxMape,0.00416913,0.000161491,0.000283634,0.00196925,0.0141745,0.173434
AllSpvCategories-withTrace-useMSE,0.00333668,0.000261425,0.000374826,0.0017149,0.013142,0.133473
